## KNN+PCA wrapper##

In [5]:
class KNNnPCAClassifier:
    def __init__(self, k, alpha):
        self.knn_classifier = metnum.KNNClassifier(k, "distance_pow")
        self.pca = PCA(alpha)

    def fit(self, X_train, y_train):
        self.pca.fit(X_train)
        self.knn_classifier.fit(self.pca.transform(X_train), y_train)
    
    def predict(self, X):
        return self.knn_classifier.predict(self.pca.transform(X))